<a href="https://colab.research.google.com/github/Satish-Kumar-1/Fine-tuning_llms/blob/main/Finetuning_llm_for_code_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip -q install transformers datasets
!pip -q install PyGithub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.2/409.2 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 40.5 MB/s eta 0:00:00


In [3]:
from github import Github
import re
from datasets import Dataset


In [4]:
from getpass import getpass

github_api_key = getpass("Enter your Github api key: ")

Enter your Github api key: ··········


In [5]:
# Initialize the github token

g = Github(github_api_key)

In [6]:
repo = g.get_repo("openai/gym")

In [7]:
def extract_functions_from_code(code):
    pattern = re.compile(r"def\s+(\w+)\s*\(.*\):")
    functions =  pattern.findall(code)
    return functions

In [8]:
# fetch files from the repo

python_files = []
contents = repo.get_contents("")

while contents:
    file_content = contents.pop(0)
    if file_content.type == "dir":
        contents.extend(repo.get_contents(file_content.path))

    elif file_content.path.endswith(".py"):
        python_files.append(file_content)



In [9]:
# extract functions and create dataset

data = {"code": [], "function_name": []}
for file in python_files:
  code = file.decoded_content.decode("utf-8")
  functions = extract_functions_from_code(code)
  for function in functions:
    data["code"].append(code)
    data["function_name"].append(function)


In [10]:
# create a hugging face dataset

dataset = Dataset.from_dict(data)

In [11]:
# save the data to disk

dataset.save_to_disk("code_generation_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/974 [00:00<?, ? examples/s]

In [12]:
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForCausalLM

In [13]:
model_checkpoint = "Salesforce/codegen-350M-mono"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/240 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/797M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/797M [00:00<?, ?B/s]

Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (e

In [14]:
tokenizer.pad_token = tokenizer.eos_token

In [23]:
from torch.utils.data import DataLoader

In [31]:


dataset = load_from_disk("code_generation_dataset")

# dataloader = DataLoader(dataset, batch_size = 16, shuffle = True )

In [32]:
# Split the dataset into train and test

dataset = dataset.train_test_split(test_size = 0.1)

In [47]:
# preprocess the dataset

def preprocess_function(examples):
    model_inputs = tokenizer(examples["code"], truncation=True, padding="max_length", max_length=256)
    model_inputs["labels"] = model_inputs["input_ids"].copy()  # CLM requires labels = input_ids
    return model_inputs


In [48]:
tokenized_datasets = dataset.map(preprocess_function, batched = True)

Map:   0%|          | 0/876 [00:00<?, ? examples/s]

Map:   0%|          | 0/98 [00:00<?, ? examples/s]

In [49]:
from transformers import TrainingArguments, Trainer

In [50]:
# fine tuned model

training_args = TrainingArguments(
    output_dir = "./results",
    per_device_train_batch_size = 8,
    num_train_epochs = 1,
    save_steps = 10000,
    save_total_limit = 2,
    fp16 = True
)

In [51]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['test']
)

In [52]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=110, training_loss=0.34646623784845526, metrics={'train_runtime': 116.1426, 'train_samples_per_second': 7.542, 'train_steps_per_second': 0.947, 'total_flos': 409424602595328.0, 'train_loss': 0.34646623784845526, 'epoch': 1.0})

In [68]:
# define a function to generate code using the fine tuned model

def generate_code(prompt, max_length = 500):
    inputs = tokenizer(prompt, return_tensors = "pt").to(model.device)
    outputs = model.generate(**inputs, max_length = max_length)
    generated_code = tokenizer.decode(outputs[0], skip_special_tokens = True)
    return generated_code





In [69]:
prompt = "def sort(arr)"
generated_code = generate_code(prompt)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [70]:
print("Generated code is: ")
print(generated_code)

Generated code is: 
def sort(arr) :
    """Sort a list in ascending order."""
    n = len(arr)
    for i in range(n-1):
        min_idx = i
        for j in range(i+1, n):
            if arr[j] < arr[min_idx]:
                min_idx = j
        arr[i], arr[min_idx] = arr[min_idx], arr[i]
    return arr

def test_sort():
    """Tests for the sort function."""
    lst = [5, 4, 3, 2, 1]
    expected_lst = [1, 2, 3, 4, 5]
    assert sort(lst) == expected_lst

def test_sort_desc():
    """Tests for the sort function in descending order."""
    lst = [5, 4, 3, 2, 1]
    expected_lst = [1, 2, 3, 4, 5]
    assert sort(lst, descending=True) == expected_lst

def test_sort_asc():
    """Tests for the sort function in ascending order."""
    lst = [5, 4, 3, 2, 1]
    expected_lst = [1, 2, 3, 4, 5]
    assert sort(lst, ascending=True) == expected_lst

def test_sort_random():
    """Tests for the sort function in a random order."""
    lst = [5, 4, 3, 2, 1]
    expected_lst = [1, 2, 3, 4, 5]
    as